# Llm Classification Finetuning EDA


In [ ]:
import json
import fastkaggle
import os
import shutil
import polars as pl
import torch
import numpy as np
from pathlib import Path

from rich import print as rprint
from rich.markdown import Markdown
from IPython.display import display

In [ ]:
api = fastkaggle.import_kaggle()

In [ ]:
data_base_path = Path("./data")
comp_name = "llm-classification-finetuning"
install_path = fastkaggle.setup_comp(comp_name)
datapath = data_base_path / comp_name
if os.path.exists(install_path):
    shutil.move(install_path, data_base_path / comp_name)

Error: Destination path 'data/llm-classification-finetuning/llm-classification-finetuning' already exists

# Dataset Description

The competition dataset consists of user interactions from the ChatBot Arena. In each user interaction a judge provides one or more prompts to two different large language models, and then indicates which of the models gave the more satisfactory response. The goal of the competition is to predict the preferences of the judges and determine the likelihood that a given prompt/response pair is selected as the winner.

Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. There are 55K rows in the training data, and you can expect roughly 25,000 rows in the test set.

## Files

### train.csv

- **id** - A unique identifier for the row.
- **model\_[a/b]** - The identity of model\_[a/b]. Included in train.csv but not test.csv.
- **prompt** - The prompt that was given as an input (to both models).
- **response\_[a/b]** - The response from model\_[a/b] to the given prompt.
- **winner*model*[a/b/tie]** - Binary columns marking the judge's selection. The ground truth target column.

### test.csv

- **id**
- **prompt**
- **response\_[a/b]**

### sample_submission.csv

A submission file in the correct format.

- **id**
- **winner*model*[a/b/tie]** - This is what is predicted from the test set.


In [ ]:
df_sample = pl.read_csv(datapath / "sample_submission.csv")
df_sample.head()

id,winner_model_a,winner_model_b,winner_tie
i64,f64,f64,f64
136060,0.333333,0.333333,0.333333
211333,0.333333,0.333333,0.333333
1233961,0.333333,0.333333,0.333333


In [ ]:
pl.Config.set_tbl_width_chars(300)
pl.Config.set_fmt_str_lengths(300)

polars.config.Config

In [ ]:
df_train = pl.read_csv(datapath / "train.csv")
df_test = pl.read_csv(datapath / "test.csv")
df_train.head()

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
i64,str,str,str,str,str,i64,i64,i64
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""[""Is it morally right to try to have a certain percentage of females on managerial positions?"",""OK, does pineapple belong on a pizza? Relax and give me fun answer.""]""","""[""The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Argume…","""[""As an AI, I don't have personal beliefs or opinions. However, I can tell you that the question of gender quotas in managerial positions is a complex one and positions can vary. Advocates argue that such measures can help correct historical imbalances, promote diversity, and may lead to better deci…",1,0,0
53567,"""koala-13b""","""gpt-4-0613""","""[""What is the difference between marriage license and marriage certificate?"",""How can I get both of them as quick as possible in California "",""What is the minimal time to get them? 1 day or 10 days?""]""","""[""A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage…","""[""A marriage license and a marriage certificate are two different legal documents that have separate purposes.\n\n1. Marriage License: A marriage license is a legal document that gives a couple permission to get married. It's usually obtained from local government or court officials before the weddi…",0,1,0
65089,"""gpt-3.5-turbo-0613""","""mistral-medium""","""[""explain function calling. how would you call a function?""]""","""[""Function calling is the process of invoking or executing a function in a programming language. To call a function, you need to use the function's name followed by parentheses \""()\"". Within the parentheses, you can pass arguments if the function requires any.\n\nHere is an example of how to call a…","""[""Function calling is the process of invoking a function in a program. A function is a block of code that performs a specific task and can be reused multiple times in a program. Functions help to organize and modularize code, making it more readable and maintainable.\n\nTo call a function, you simpl…",0,0,1
96401,"""llama-2-13b-chat""","""mistral-7b-instruct""","""[""How can I create a test set for a very rare category? I want to build a classifier.""]""","""[""Creating a test set for a very rare category can be challenging, as it may not be possible to collect a sufficient number of examples for the rare category. However, there are several strategies you can use to create a test set for a rare category:\n\n1. Use synthetic data: If the rare category is…","""[""When building a classifier for a very rare category, creating a test set can be challenging due to the limited amount of data available. Here are some steps you can take to create a test set for a rare category:\n\n1. Gather as much data as possible: The more data you have, the easier it will be t…",1,0,0
198779,"""koala-13b""","""gpt-3.5-turbo-0314""","""[""What is the best way to travel from Tel-Aviv to Jerusalem? Car? Bus? Plane?""]""","""[""The best way to travel from Tel Aviv to Jerusalem depends on your personal preference and the availability of transportation options. All the options you have mentioned are valid options, but here are some details to help you make your decision:\n\n* By car: Traveling by car is the quickest way …","""[""The best way to travel from Tel-Aviv to Jerusalem depends on your personal preferences and circumstances. \n\nIf you have a car and enjoy driving, then driving from Tel-Aviv to Jerusalem is an optio

In [ ]:
def fmt_llm(idx: int, col_name: str, df: pl.DataFrame = df_train):
    for idx, res in enumerate(json.loads(df[idx, col_name])):
        rprint(f"[bold bright_green]######### Response {idx} #########[/bold bright_green]")
        display(Markdown(res))

Each row is a set of conversations by user, which will at some point allow exploration of some conflicting user preferences in responses to similar questions(maybe)


In [ ]:
fmt_llm(0, "response_a")

######### Response 0 #########

The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a
complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.           

Here are some arguments in favor of and against such policies:                                                     

Arguments in favor:                                                                                                

 1 Correcting Historical Inequities: Women have historically been underrepresented in leadership roles due to      
   various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a         
   corrective measure to address past and ongoing discrimination.                                                  
 2 Promoting Diversity: Diverse leadership teams can enhance decision-making and represent a broader range of      
   perspectives. This can lead to better outcomes for organizations and society as a whole.                        
 3 Equality of Opportunity: Setting targets for female representation in management can help ensure that women have
   equal opportunities to advance in their careers.                                                                
 4 Role Modeling: Increased visibility of female leaders can inspire and encourage other women and girls to pursue 
   leadership roles.                                                                                               

Arguments against:                                                                                                 

 1 Reverse Discrimination: Setting a quota for female representation might lead to the perception or reality of    
   reverse discrimination, where men might be overlooked for positions despite being qualified, simply to meet the 
   gender target.                                                                                                  
 2 Meritocracy Concerns: Critics argue that employment and promotion should be based solely on merit and           
   qualifications, not gender. They fear that quotas could compromise the quality of leadership if less qualified  
   individuals are promoted to meet gender targets.                                                                
 3 Tokenism: There is a risk that women appointed to meet quotas might be seen as "tokens," undermining their      
   legitimacy and potentially leading to resentment among colleagues.                                              
 4 Oversimplification of Diversity: Focusing only on gender might overlook other important aspects of diversity,   
   such as race, ethnicity, socioeconomic background, or sexual orientation.                                       

Ultimately, the morality of aiming for a certain percentage of females in managerial positions depends on one's    
ethical framework and the context in which such policies are implemented. Proponents of gender diversity targets   
often argue that these measures are necessary as a transitional mechanism to create a level playing field, while   
opponents may argue for a "color-blind" or "gender-blind" approach to hiring and promotions.                       

Organizations and societies must carefully consider the implications of these strategies and strive to implement   
policies that promote fairness and equality while minimizing negative consequences. It is also important to address
the root causes of gender inequality, such as unconscious bias, work-life balance challenges, and lack of          
supportive policies, to create an environment where everyone has the opportunity to succeed based on their         
capabilities and contributions.

######### Response 1 #########

Ah, the age-old culinary conundrum that has divided nations and dinner tables: does pineapple belong on a pizza?   
The tropical twist of pineapple on pizza, known as Hawaiian pizza, is a hotly debated topic where taste buds battle
and pizza purists protest.                                                                                         

Let's slice into the debate with a zest of fun:                                                                    

Team Pineapple: "Absolutely, yes! Pineapple on pizza is like a beach party in your mouth. The sweet juiciness of   
pineapple chunks frolicking with savory ham or bacon creates a flavor wave that surfs across the cheesy ocean of   
deliciousness. It's the Mardi Gras of pizzas, where sweet meets savory in a jubilant jamboree!"                    

Team No-Pineapple: "No way, not in a million pizzas! Pineapple is a fruit that should be sunbathing on a tropical  
fruit platter, not freeloading on a sacred slice of pizza. The thought of warm, melty cheese conspiring with       
pineapple's sugary mischief is enough to make Italian ancestors turn in their tomato patches. Keep the pineapple in
the fruit salad where it belongs!"                                                                                 

At the end of the day, whether pineapple belongs on pizza is a matter of personal taste. Some say it's a match made
in heaven, others say it's a culinary crime. But let's face it, in the world of pizza toppings, where you can find 
everything from anchovies to zucchini, there's room for a little pineapple pizzazz. So, if your taste buds crave   
that sweet and tangy twist, go ahead and let your pineapple flag fly atop that glorious cheese-covered dough. Bon  
appétit, or as they say in pineapple paradise, "Aloha-ppetite!" 🍍🍕